<a href="https://colab.research.google.com/github/keim-hs-esslingen/MicroChargeNet/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install sumolib
!pip install pyproj
!pip install rtree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import sumolib
import pyproj
import numpy as np

import matplotlib.pyplot as plt
# parse the net

net = sumolib.net.readNet('drive/MyDrive/Colab Notebooks/Stuttgart/stuttgart.net.xml.gz')

x0, y0, x1, y1 = net.getBoundary()


# create Mesh
count=10
xs = np. linspace(x0, x1, count)
ys = np. linspace(y0, y1, count)
X, Y = np. meshgrid(xs, ys)

# parse Coordinates 
rawPoints = np.append(X.reshape(-1,1),Y.reshape(-1,1),axis=1)

#landuse

polys = []

for poly in sumolib.shapes.polygon.read('drive/MyDrive/Colab Notebooks/Stuttgart/stuttgart.poly.xml'):
  if poly.type.startswith('landuse.industrial'):
    polys.append(poly)

print(polys[0])
print(sumolib.geomhelper.isWithin((4070, 2440), polys[0].shape))

###

"""
# Plot coordinates
plt.figure(figsize=(10,10), dpi=80)

plt.plot(X, Y, '.', color='black');
plt.show()
"""

allowedEdgeTypes = {"highway.primary", "highway.primary_link", "highway.secondary", "highway.secondary_link",
                       "highway.tertiary", "highway.tertiary_link", "highway.unclassified", "highway.residential",
                       "highway.living_street", "highway.cycleway" }
allowedVehicleClasses = set(['private', 'bicycle', 'moped', 'motorcycle', 'evehicle'])

# find nearest Edge
radius = x1/count
lanes = net.getNeighboringLanes(5168, 1513, radius, includeJunctions=False)
# pick the closest edge
nearestLane = None
if len(lanes) > 0:
    clearedLanes = []

    for e in lanes:
      if e[0].getEdge().getType() in allowedEdgeTypes:
        if (e[0]._allowed & allowedVehicleClasses):
          clearedLanes.append(e)

    laneAndDistance = sorted(clearedLanes, key=lambda x: x[1] )
    nearestLane = laneAndDistance[0]

print(nearestLane[0].getID())
xPos, yPos = nearestLane[0].getShape()[0]
print(xPos)
print(yPos)


<poly id="1012759231" type="landuse.industrial" color="209.0,209.0,204.0" layer="-3" fill="1" shape="3607.02137,3047.385018 3600.735468,3037.165547 3599.073515,3031.70359 3596.895993,3026.818458 3585.221273,3009.071772 3579.057155,3000.130972 3577.070436,2995.312995 3576.440122,2991.587596 3578.184019,2987.223051 3581.994029,2982.040779 3592.512684,2972.950317 3601.544052,2965.323698 3623.348581,2948.945499 3625.9024,2946.917277 3630.45683,2953.141978 3624.256667,2962.564943 3623.000667,2968.720328 3624.008549,2974.303023 3614.006525,2979.237202 3591.091511,2996.213093 3587.906706,3001.052232 3590.549782,3004.77126 3597.787385,3016.3936 3611.997984,3033.445982 3615.826995,3038.879712 3607.02137,3047.385018"/>
False


'\n# Plot coordinates\nplt.figure(figsize=(10,10), dpi=80)\n\nplt.plot(X, Y, \'.\', color=\'black\');\nplt.show()\n\n\nallowedEdgeTypes = {"highway.primary", "highway.primary_link", "highway.secondary", "highway.secondary_link",\n                       "highway.tertiary", "highway.tertiary_link", "highway.unclassified", "highway.residential",\n                       "highway.living_street", "highway.cycleway" }\nallowedVehicleClasses = set([\'private\', \'bicycle\', \'moped\', \'motorcycle\', \'evehicle\'])\n\n# find nearest Edge\nradius = x1/count\nlanes = net.getNeighboringLanes(5168, 1513, radius, includeJunctions=False)\n# pick the closest edge\nnearestLane = None\nif len(lanes) > 0:\n    clearedLanes = []\n\n    for e in lanes:\n      if e[0].getEdge().getType() in allowedEdgeTypes:\n        if (e[0]._allowed & allowedVehicleClasses):\n          clearedLanes.append(e)\n\n    laneAndDistance = sorted(clearedLanes, key=lambda x: x[1] )\n    nearestLane = laneAndDistance[0]\n\nprint(